In [1]:
ls

NewMetrics.ipynb  README.md         practice2.ipynb
ParkFactor.ipynb  Setup.ipynb


In [2]:
cd ../sabermetrics

/Users/namshik/Documents/py/baseball_lab/src/sabermetrics



Other park correctors in baseball-reference considers the correction of the initial factor 

Initial factor of a park is computed by (Scored RUNs of home team at home per home game number + Allowed RUNs of home team at home per home game number )/ (SR of HT at away per AGN + AR of HT at away per AGN ). In other words, it presents how easy to make and allow runs at home.

The rough park factor can be calculated by (initial factor -1)/2 + 1. For example, if initial factor is 1.2, the rough park factor not considering small corrections is 1.1.

I recommend you to read the [this link](https://www.baseball-reference.com/about/parkadjust.shtml) before further discussion. The above link introduce how to compute the park factor for baseball reference. 

I especially focused on the step 3, OPC (other park corrector).

"Make corrections for the fact that the other road parks' total difference from the league average is offset by the park rating of the club that is being rated"

BTW, it assumes the initial factors of the other parks are all identical, or have the same number of games with N-1 teams (N=30).

More details with formula will be presented at [my personal blog](http://physhik.com/opc/).

However, it's slighty off because a mlb team have more games in the same division. This was already pointed out in other posts such as [this](https://www.reddit.com/r/Sabermetrics/comments/4kjlw9/are_park_factors_real/) 

I calculated the more precise OPC x Initial factor from the number of all the games of all the teams by web crawling mlb schedule.

I also updated for baseball-reference 'Pitching/batting park factor(PPF/BPF)' via 'Team pitching/batting rating(TPR/TBR)'. The formula to this also seems to have room to be improved to me, and still on progressing. 

The park factor in baseball reference and fangraphs are scaled separatedly for NL/AL. I did not split the league, and DH/non-DH effect was not considered yet. Thus, the resultant park factor I calucuated overestimates AL yet. I will modify this soon. 

In [3]:
from parkfactor import *

Now we need the records of GP (games played), W at home and away for each team and park. 


You need to run gamestadium function, but it takes much time. If you are interested in 2014 ~ 2018, I saved the dictionary in /downloads directory, and you can load the data. 

In [0]:
# Use the saved data 

import json 
year = 2018

with open('../../baseball/teamd'+str(year)+'.json') as json_file:
    teamd = json.load(json_file)

In [0]:
with open('teamd2018.json', 'w') as outfile:
    json.dump(teamd, outfile)

In [ ]:
def gamestadium1(year, teaml, teamd):
  """
  Make a dictionary data structure has team names keys and game numbers played at each parks as home/away.
  """
  url = "https://www.espn.com/mlb/team/schedule/_/name/{}/season/{}/seasontype/2/half/{}"
    
  for t in teaml: 

      for i in [2,1]:
        url_half = url.format(t,year, i)
        s=requests.get(url_half)
        soup=BeautifulSoup(s.content,'lxml')
        table = soup.find_all('tr')
        data = []
        for ta in table:
          cols = ta.find_all('td')

          cols = [ele.text.strip() for ele in cols]
          try:
            teamname = ta.find('a').get('href')
            full = {('mets',' Mets'),( 'yankees', ' Yankees'), ('white', ' White Sox'),('cubs', ' Cubs'), ('angels', ' Angels'), ('dodgers', ' Dodgers')}
            for i, j in full:
              if i in teamname:
                cols[1] += j
          except:
            pass
          
          data.append([ele for ele in cols[1:]])
    
        if data !=[] :
          dp = pd.DataFrame(data[1:])
          dp.columns =  data[0]
          
        
          
          # remove postpone date using None
          dp= dp.iloc[:,0:5]
          dp = dp.dropna()
          print(dp.columns)
          for i in range(len(dp)):
            team  = dp.Opponent.values[i]
            result = dp.Result.values[i]
            result = result.split(' ')[0]
            WL = result[0]
            #print(WL)
            scores = result[1:].split('-')
            
            def addscore(ha, b):
              
              try:
                teamd[t][ha][team+' SR'] += int(scores[b])
                teamd[t][ha][team+' AR'] += int(scores[(b+1)%2])
              except:
                teamd[t][ha][team+' SR'] = int(scores[b])
                teamd[t][ha][team+' AR'] = int(scores[(b+1)%2])
                       
            if team[0] == 'v' and (WL in 'WL'):
              team = team[3:]        
              try:
                teamd[t]['Home'][team+' GP'] +=1
              except:            
                teamd[t]['Home'][team+' GP'] =1

               
              if WL == 'W':
                addscore('Home', 0)
              else:
                addscore('Home',1)
                  
            elif team[0] =='@' and (WL in 'WL'):
              team = team[2:]
              try:
                teamd[t]['Away'][team+' GP'] +=1
              except:            
                teamd[t]['Away'][team+' GP'] =1
              if WL == 'W':  
                addscore('Away', 0)
              else:
                addscore('Away',1)

  teaml=[]
  for t in teamd.keys():
    if teamd[t] == {'Away': {}, 'Home': {}}:
      teaml.append(t) 
  
  if teaml == []:
    return teamd
  print(len(teaml), " teams are on processing...")
  
  gamestadium(year,teaml, teamd)      

  return teamd

def solver(year, IF, teamd):
  "Find opc x runs by solving linear equations, and return the joined dataframe."
  
  year = str(year)  
  tmap = {'ari': 'Arizona',
 'atl': 'Atlanta',
 'bal': 'Baltimore',
 'bos': 'Boston',
 'chc': 'Chicago Cubs',
 'chw': 'Chicago White Sox',
 'cin': 'Cincinnati',
 'cle': 'Cleveland',
 'col': 'Colorado',
 'det': 'Detroit',
 'hou': 'Houston',
 'kc': 'Kansas City',
 'laa': 'Los Angeles Angels',
 'lad': 'Los Angeles Dodgers',
 'mia': 'Miami',
 'mil': 'Milwaukee',
 'min': 'Minnesota',
 'nym': 'New York Mets',
 'nyy': 'New York Yankees',
 'oak': 'Oakland',
 'phi': 'Philadelphia',
 'pit': 'Pittsburgh',
 'sd': 'San Diego',
 'sea': 'Seattle',
 'sf': 'San Francisco',
 'stl': 'St. Louis',
 'tb': 'Tampa Bay',
 'tex': 'Texas',
 'tor': 'Toronto',
 'wsh': 'Washington'}
  
  teaml = list(tmap.keys())
  
  pol = [[0 for x in range(30)] for y in range(30)]
  
  poltr = [[0 for x in range(60)] for y in range(60)]

  for i in range(30):
    s=0
    h = teamd[teaml[i]]['Home']
    a = teamd[teaml[i]]['Away']

    for j in range(30):
      if tmap[teaml[j]]+' GP' in h.keys():
        s += h[tmap[teaml[j]]+ ' GP']    

      if tmap[teaml[j]]+' GP' in a.keys():
        pol[i][j] = a[tmap[teaml[j]] + ' GP']
    pol[i][i] =s    

  for i in range(30):
    for j in range(30):
      if i!=j:
        pol[i][j] *= -float(IF.loc[tmap[teaml[i]]][year])
  
  
  #for i in range(30):
   # for j in range(30):
    #  poltr[i][j]=IF.loc[tmap[teaml[i]]]['SF']*
  
  
  print("The determinant of 30x30 matrice must be near zero, ", np.linalg.det(pol)/(80**30))
  
  x= [0]*30
  xl = []
  for i in range(30):
    pol2=pol[:][:]


    pol2[29-i] = [1]*30
    A= np.array(pol2)
    b= [0]*30
    b[29-i] = 30
    b = np.array(b)
    x += np.linalg.solve(A,b)
    xl.append(np.linalg.solve(A,b))

  SF = x/30 / IF[year+'IPC']


  print("Ideally sum of the factors is 30, and here is ",sum(SF))
  d = {'mOPCxRUN/IPC':[]}
  for i in range(30):
    d['mOPCxRUN/IPC'].append(SF[i])  
  dp = pd.DataFrame(d)
  dp.index = IF.index
  
  return IF.join(dp)

In [30]:
teamd = gamestadium1(year, teaml, teamd)


                  0       1      2               3               4  \
0          Opponent  Result    W-L             WIN            LOSS   
1       @ Milwaukee    W6-4  54-43       Hill  3-4   Williams  0-3   
2       @ Milwaukee    L4-2  54-44     Burnes  1-0    Kershaw  3-5   
3       @ Milwaukee   W11-2  55-44       Wood  6-5      Suter  8-7   
4    @ Philadelphia    W7-6  56-44  Alexander  2-0  Dominguez  1-3   
..              ...     ...    ...             ...             ...   
63        @ Arizona    L7-2  88-71  Greinke  15-11  Stripling  8-6   
64  @ San Francisco    W3-1  89-71        Ryu  7-3  Bumgarner  6-7   
65  @ San Francisco   W10-6  90-71       Wood  9-7   Melancon  1-4   
66  @ San Francisco   W15-0  91-71      Hill  11-5    Suarez  7-13   
67      vs Colorado    W5-2  92-71    Buehler  8-5  Marquez  14-11   

              5       6  
0          SAVE     ATT  
1    Jansen  28  36,812  
2    Knebel  12  36,242  
3   Ferguson  2  38,249  
4    Jansen  29  33,753  
.. 

                         0          1      2               3               4  \
0                 Opponent     Result    W-L             WIN            LOSS   
1              vs Colorado     L11-10  53-45      Oberg  5-0    Bradley  2-2   
2              vs Colorado       L6-5  53-46      Oberg  6-0     Hirano  2-2   
3              vs Colorado       W6-1  54-46   Greinke  11-5  Senzatela  3-3   
4           @ Chicago Cubs       W7-1  55-46     Corbin  7-4    Farrell  3-4   
..                     ...        ...    ...             ...             ...   
61  vs Los Angeles Dodgers       W4-3  80-78  Boxberger  3-7     Maeda  8-10   
62  vs Los Angeles Dodgers       W7-2  81-78  Greinke  15-11  Stripling  8-6   
63             @ San Diego  L3-2 F/15  81-79     Brewer  1-0   Andriese  0-3   
64             @ San Diego       W5-4  82-79   Godley  15-11        Nix  2-5   
65             @ San Diego  L4-3 F/10  82-80   Castillo  3-3    Barrett  0-1   

              5       6  
0          SA

                 0          1      2              3                4  \
0         Opponent     Result    W-L            WIN             LOSS   
1    vs Pittsburgh      L12-1  43-54   Taillon  7-7       Mahle  7-8   
2    vs Pittsburgh       L6-2  43-55   Kingham  5-4  DeSclafani  4-3   
3    vs Pittsburgh       L9-2  43-56      Nova  6-6      Harvey  5-6   
4     vs St. Louis       W2-1  44-56    Hughes  3-3      Norris  3-3   
..             ...        ...    ...            ...              ...   
63  vs Kansas City       L4-3  66-92  McCarthy  5-4    Iglesias  2-5   
64  vs Kansas City       L6-1  66-93  Fillmyer  4-2        Reed  1-3   
65   vs Pittsburgh       L8-4  66-94    Brault  6-3  DeSclafani  7-8   
66   vs Pittsburgh       W3-0  67-94  Lorenzen  4-2   Taillon  14-10   
67   vs Pittsburgh  L6-5 F/10  67-95     Feliz  1-2    Stephens  2-3   

               5       6  
0           SAVE     ATT  
1                 20,726  
2                 23,244  
3                 23,615  


               0          1      2                   3               4  \
0       Opponent     Result    W-L                 WIN            LOSS   
1   vs San Diego      W11-5  54-42          Davis  1-0    Richard  7-9   
2   vs San Diego  Postponed   None                None            None   
3   vs San Diego      L10-2  54-43           Ross  6-8    Pivetta  6-8   
4   vs San Diego       W5-0  55-43      Velasquez  6-8    Perdomo  1-5   
..           ...        ...    ...                 ...             ...   
65    @ Colorado      L14-0  78-80      Marquez  14-10   Pivetta  7-14   
66    @ Colorado       L5-3  78-81          Oberg  8-1  Arrieta  10-11   
67    vs Atlanta      L10-2  78-82  Foltynewicz  13-10   Eickhoff  0-1   
68    vs Atlanta       W3-0  79-82          Nola  17-6    Venters  4-1   
69    vs Atlanta       W3-1  80-82         Hunter  5-4    Gausman  5-3   

                5       6  
0            SAVE     ATT  
1                  30,034  
2            None    None  

                       0       1      2               3                4  \
0               Opponent  Result    W-L             WIN             LOSS   
1   vs Chicago White Sox    W3-1  59-39    LeBlanc  6-1    Shields  4-11   
2   vs Chicago White Sox    L5-0  59-40      Covey  4-5   Hernandez  8-8   
3   vs Chicago White Sox    W8-2  60-40  Gonzales  11-5       Lopez  4-8   
4       vs San Francisco    L4-3  60-41     Watson  3-3        Diaz  0-3   
..                   ...     ...    ...             ...              ...   
61            vs Oakland    L9-3  86-72    Buchter  6-0  Hernandez  8-14   
62              vs Texas    L2-0  86-73     Jurado  5-5        Duke  5-5   
63              vs Texas   W12-6  87-73    LeBlanc  9-5       Perez  2-7   
64              vs Texas    W4-1  88-73    Vincent  4-4     Sampson  0-3   
65              vs Texas    W3-1  89-73      Elias  3-1    Gallardo  8-8   

               5       6  
0           SAVE     ATT  
1       Diaz  37  43,331  
2     

                   0          1       2                 3                4  \
0           Opponent     Result     W-L               WIN             LOSS   
1   vs New York Mets       L7-5   62-34  Syndergaard  6-1      German  2-6   
2   vs New York Mets       W7-6   63-34         Gray  7-7        Matz  4-8   
3   vs New York Mets  Postponed    None              None             None   
4        @ Tampa Bay       L7-6   63-35     Andriese  3-4   Severino  14-3   
..               ...        ...     ...               ...              ...   
65       @ Tampa Bay       L8-7   97-61     Chirinos  5-5     Tanaka  12-6   
66       @ Tampa Bay      W12-1   98-61     Sabathia  9-7     Schultz  2-2   
67          @ Boston      W11-6   99-61        Happ  17-6     Johnson  4-5   
68          @ Boston       W8-5  100-61       Lynn  10-10  Rodriguez  12-5   
69          @ Boston      L10-2  100-62   Rodriguez  13-5       Cessa  1-4   

              5       6  
0          SAVE     ATT  
1   Gsellma

                      0       1      2                3                 4  \
0              Opponent  Result    W-L              WIN              LOSS   
1              vs Miami    L6-5  49-48     Straily  4-4     Castillo  1-1   
2              vs Miami    L3-2  49-49       Lopez  2-1    Yarbrough  8-5   
3              vs Miami    W6-4  50-49     Kolarek  1-0  Barraclough  0-4   
4   vs New York Yankees    W7-6  51-49    Andriese  3-4    Severino  14-3   
..                  ...     ...    ...              ...               ...   
62  vs New York Yankees    W8-7  88-70    Chirinos  5-5      Tanaka  12-6   
63  vs New York Yankees   L12-1  88-71    Sabathia  9-7      Schultz  2-2   
64           vs Toronto    L7-6  88-72       Mayza  2-0     Alvarado  1-6   
65           vs Toronto    W4-3  89-72    Castillo  4-2      Borucki  4-6   
66           vs Toronto    W9-4  90-72  Yarbrough  16-6    Gaviglio  3-10   

                  5       6  
0              SAVE     ATT  
1         Conle

ValueError: Length mismatch: Expected axis has 7 elements, new values have 2 elements

In [5]:
# make your own 

year = 2018 

teamd = {}
teaml = ['lad','col','sf','ari','sd','stl','pit','cin','chc','mil','phi','nym','wsh','mia','atl','hou','tex','oak','sea','laa','min','kc','cle','chw','det','nyy', 'tor','bos','tb','bal'] 

for t in teaml:
  teamd[t] = {'Home' :{}, 'Away':{}}

teamd = gamestadium(year, teaml, teamd)


AttributeError: 'DataFrame' object has no attribute 'Opponent'

In [29]:
pf = oldBP(2018,teamd,True)

pf

KeyError: 0

In [28]:
teamd

{'lad': {'Home': {'Milwaukee GP': 4,
   'Milwaukee SR': 29,
   'Milwaukee AR': 15,
   'Houston GP': 3,
   'Houston SR': 4,
   'Houston AR': 18,
   'San Francisco GP': 10,
   'San Francisco SR': 28,
   'San Francisco AR': 19,
   'St. Louis GP': 3,
   'St. Louis SR': 6,
   'St. Louis AR': 13,
   'San Diego GP': 9,
   'San Diego SR': 62,
   'San Diego AR': 24,
   'Arizona GP': 9,
   'Arizona SR': 36,
   'Arizona AR': 39,
   'New York Mets GP': 3,
   'New York Mets SR': 16,
   'New York Mets AR': 15,
   'Colorado GP': 10,
   'Colorado SR': 38,
   'Colorado AR': 23,
   'Oakland GP': 2,
   'Oakland SR': 10,
   'Oakland AR': 16,
   'Washington GP': 3,
   'Washington SR': 10,
   'Washington AR': 8,
   'Miami GP': 3,
   'Miami SR': 10,
   'Miami AR': 12,
   'Cincinnati GP': 4,
   'Cincinnati SR': 9,
   'Cincinnati AR': 20,
   'Philadelphia GP': 4,
   'Philadelphia SR': 15,
   'Philadelphia AR': 14,
   'Atlanta GP': 3,
   'Atlanta SR': 17,
   'Atlanta AR': 10,
   'Texas GP': 2,
   'Texas SR': 15